In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import einops
import torch
import wandb
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env
from wandb.integration.sb3 import WandbCallback
import plotly.express as px

import src as M

# For plotly, to have larger images
BIG = dict(width=1600, height=1600)

In [ ]:
red, green, blue = M.ThreeGoalsEnv.GOAL_CELLS
env = lambda: (
    M.ColorBlindWrapper.merged(
        M.AddTrueGoalWrapper(
            M.ThreeGoalsEnv()
        ),
        red, green
    )
)

check_env(env())
print(env())
print(env().observation_space)
print(env().action_space)
o, _ = env().reset()
switch = o['switch']
print(f"{switch=}")
obs = o['obs']
print(obs[:80].astype(int).reshape(4, 4, -1))
print(obs[80:].astype(int))



In [ ]:
net = M.SwitchMLP(4, (64, 64), 2, 3, 0)
print(net)

In [ ]:
x = torch.tensor([
    [1.0, 2, 3, 4],
    [4.0, 3, 2, 1],
])
switch_idx = [0, 2]
switch = torch.zeros((2, 3))
switch[range(2), switch_idx] = 1

print(x)
print(switch)
print(net(x, switch))


In [ ]:
def unique(x, *, __previous=set()):
    """Return the argument, if it was never seen before, otherwise raise ValueError"""
    if x in __previous:
        raise ValueError(f"Duplicate value {x}")
    __previous.add(x)
    return x

class WandbWithBehaviorCallback(WandbCallback):
    def __init__(self, show_every=10, **kwargs):
        self.show_every = show_every
        self.time = 0
        super().__init__(**kwargs)
    def _on_rollout_start(self) -> None:
        super()._on_rollout_start()
        # Show every  10 episodes
        self.time += 1
        if self.time % self.show_every == 0:
            M.show_behavior(self.model, env(), max_len=20, add_to_wandb=True, plot=False)

In [ ]:
env_size = 4
red, green, blue = M.ThreeGoalsEnv.GOAL_CELLS
env = lambda: (
    M.ColorBlindWrapper.merged(
        M.AddTrueGoalWrapper(
            M.ThreeGoalsEnv(env_size)
        ),
        red,
        # green,
    )
)
env = lambda: (
    M.MergeGoalWithObsWrapper(
        M.ColorBlindWrapper(
            M.AddTrueGoalWrapper(
                M.ThreeGoalsEnv(env_size)
            ),
            1,
            lambda _switch: (0, [0, 1])
        )
    )
)

n_env = 4
lr_start, lr_end = 5e-3, 2e-5
policy = PPO(
    # "MlpPolicy",
    M.SwitchActorCriticPolicy,
    make_vec_env(env, n_envs=n_env, seed=42),
    policy_kwargs=dict(
        arch_kwargs=dict(
            switched_layer=0,
            hidden=[64, 32, 32],
            out_dim=32,
            n_switches=env().n_scenarios,
            # l1_reg=3e-5,
        ),
    ),
    n_steps=2_048 // n_env,
    tensorboard_log="run_logs",
    # device='cuda:1'
)
print(policy.policy)

wandb.init(
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    save_code=True,
    config=dict(
        env_size=env_size,
        lr_start=lr_start,
        lr_end=lr_end,
        arch=policy.policy_kwargs['arch_kwargs']
    ),
    project="switched-3goals-blind",
    notes=unique("""
Focus: find an agent that generalises in two different way.

red/green blind, only one agent (fix bug ×2)
""")
)

policy.learn(total_timesteps=600_000, callback=WandbWithBehaviorCallback())

In [ ]:
policy.lr_schedule = lambda _: 1e-6
policy.learn(total_timesteps=100_000, reset_num_timesteps=False, callback=WandbWithBehaviorCallback())

In [ ]:
# M.show_behavior(policy, env(), width=1600, height=1600, max_len=20
                # add_to_wandb=True,
                # )
    
wrappers = [
    M.AddTrueGoalWrapper,
    lambda e: M.ColorBlindWrapper.merged(e, red, disabled=True)
]
M.show_behavior(policy, M.ThreeGoalsEnv.interesting(4, 10, wrappers), **BIG)

In [ ]:
def add_wrappers(e, disabled):
    e = M.AddTrueGoalWrapper(e)
    e = M.ColorBlindWrapper.merged(
        e,
        red, 
        # green,
        disabled=disabled)
    return e

envs = []
for i in range(6):
    base_env = M.ThreeGoalsEnv.constant(env_size, true_goal={"red": 1, "green": 1})
    # env_blind = add_wrappers(base_env, disabled=False)
    # envs.append(env_blind)            [(0, 1), (1, 1), (0, 2)], 
    env_color = add_wrappers(base_env, disabled=True)
    envs.append(env_color)
        
M.show_behavior(policy, envs, **BIG)

In [ ]:
from tqdm.notebook import tqdm
import heapq

def evaluate(policy_, env_: M.ThreeGoalsEnv, n_episodes=1000, max_len=20, show_n=20):
    """Return the proportion of episodes where the agent reached the true goal."""
    found = 0
    wrong_goal = 0
    fails = []
    longest_successes = []
    for _ in tqdm(range(n_episodes)):
        trajectory = M.Trajectory.from_policy(
            policy_, env_, max_len=max_len, 
            end_condition=lambda locals_: env_.agent_pos == env_.goal_positions[env_.true_goal_idx]
        )
        if trajectory.ended == "condition":
            found += 1
            longest_successes.append(trajectory)
        elif len(fails) < show_n:
            fails.append(trajectory)
        if trajectory.ended == "terminated":
            wrong_goal += 1
            
    if show_n > 0:
        longest_successes.sort(key=len, reverse=True)
        longest_successes = longest_successes[:show_n]
        M.show_behavior(policy_, longest_successes + fails)
        
    return {
        "Correct goal": found / n_episodes,
        "Wrong goal": wrong_goal / n_episodes,
        "No goal": (n_episodes - found - wrong_goal) / n_episodes,
    }

evaluate(policy, env())

In [ ]:
# Print all models
models_dir = Path("models")
for model_path in sorted(models_dir.glob("*.zip")):
    print(model_path.name)

In [ ]:
name = "switched-blind=rg-4x4-switch=first-L1=1e-5"
path = models_dir / f"{name}.zip"
if path.exists():
    print("Loading existing model")
    previous_policy = policy  # Saved, in case I wanted to save it, but forgot to change the name
    policy = PPO.load(path)
else:
    print(f"Saving model to {path}")
    policy.save(path)

# Visualize the weights

In [ ]:
def imshow(x, symetric: bool = True, **kwargs):
    h, w = x.shape[-2:]
    if 'facet_col' in kwargs:
        wrap = kwargs.get('facet_col_wrap', 1)
        h *= np.ceil(x.shape[kwargs['facet_col']] / wrap)
        w *= wrap
    if symetric:
        kwargs.setdefault("color_continuous_midpoint", 0)
        kwargs.setdefault("color_continuous_scale", "RdBu")
    width = 50 + w * 25
    height = h * 25 + 50 * ('title' in kwargs)
    while width < 500 and height < 500:
        width *= 2
        height *= 2
    new = dict(
        width=max(300, width),
        height=max(300, height),
        facet_row_spacing=0.01,
        facet_col_spacing=0.01,
    )
    kwargs = {**new, **kwargs}
    px.imshow(x, **kwargs).show()


switch_biases = policy.policy.mlp_extractor.switch_biases
print("Biases shape:", switch_biases.shape)
print("Max abs bias:", switch_biases.abs().max(dim=-1).values)
imshow(switch_biases, title='Biases of the switch layers')

In [ ]:
# Plot the three switch layers

switch_layers_weights = policy.policy.mlp_extractor.switch_weights

# shape of a switch (out_dim, row, col, obj_type)
w1 = einops.rearrange(switch_layers_weights, 'agent out_dim (row col obj_type) -> agent obj_type out_dim row col', row=4, col=4)
b1 = einops.repeat(policy.policy.mlp_extractor.switch_biases, 'agent out_dim -> agent 1 out_dim 4 1')

avg = w1.mean(dim=0)
TYPES = ['empty', 'agent', 'goal_red', 'goal_green', 'goal_blue']
EMPTY, AGENT, RED, GREEN, BLUE = range(5)

print(w1.shape, avg.shape)
# d = w1[:, EMPTY] - avg[EMPTY] 
d = avg

d = w1
print(d.shape)
# Add one black col
d = torch.cat([d, torch.zeros(*d.shape[:-1], 1) + float('nan')], dim=-1)
d = torch.cat([d, torch.zeros(*d.shape[:-2], 1, d.shape[-1]) + float('nan')], dim=-2)
d = einops.rearrange(d, 'agent obj out row col -> out (agent row) (obj col)')[..., :-1, :]
b1 = torch.cat([b1, torch.zeros(*b1.shape[:-2], 1, b1.shape[-1]) + float('nan')], dim=-2)
b1 = einops.rearrange(b1, 'agent obj out row col -> out (agent row) (obj col)')[..., :-1, :]
print(d.shape, b1.shape)
d = torch.cat([d, b1], dim=-1)
# Remove weights close to zero
# d[abs(d) < 0.1] = float('nan')
imshow(d[:16], title='First layer weights', facet_col=0, facet_col_wrap=4,
          # height=4000,
          # width=None,
          )


In [ ]:
imshow(w1.flatten(2).mean(dim=2),
          title="Mean of the weights of the switch layers",
          labels=dict(x="Object type", y="Agent"),
       symetric=False,
          )
imshow(w1.flatten(2).abs().mean(dim=2), 
          title="Mean absolute value of the weights of the switch layers",
          labels=dict(x="Object type", y="Agent"),
       symetric=False,
          )
imshow(w1.flatten(2).std(dim=2),
            title="Std of the weights of the switch layers",
            labels=dict(x="Object type", y="Agent"),
       symetric=False,
          )


In [ ]:
last_layer: torch.nn.Linear = policy.policy.action_net
last_weights = last_layer.weight.detach().cpu().clone()
last_bias = last_layer.bias.detach().cpu().clone()

weights = torch.cat([
    last_weights.T @ net.switches[i].weight.detach().cpu().clone()
    for i in range(3)
], dim=0)
imshow(weights)

biases = torch.stack([net.switches[i].bias.detach().cpu() for i in range(3)], dim=1)
imshow(biases)

In [ ]:
# compute correlations between rows of w2
w2 = net.post_switch[1].weight.detach().cpu().clone()  # (64, 64)
imshow(w2)
w2 = w2 / w2.norm(dim=1, keepdim=True)
corr = w2 @ w2.T

# Cluster the correlations matrix
import scipy.cluster.hierarchy as sch
import scipy.spatial.distance as ssd
import plotly.figure_factory as ff

# Compute and plot first dendrogram.
fig = ff.create_dendrogram(
    corr.numpy(),
    orientation='left',
    labels=list(range(64)),
    linkagefun=lambda x: sch.linkage(x, 'single'),
    distfun=lambda x: ssd.pdist(x, 'euclidean'),
)
fig.update_layout(width=1000, height=1000)
fig.show()

# Remove the diagonal
corr[range(64), range(64)] = float('nan')
px.imshow(corr, width=1000, height=1000)

# Looking at the activations

In [ ]:
from hooks import record_activations

envs: list[M.ThreeGoalsEnv]
n = 5000
envs = [add_wrappers(M.ThreeGoalsEnv.constant(), disabled=False) for _ in range(n)]
inputs = [env.reset()[0] for env in envs]
with record_activations(policy.policy) as cache:
    for i in inputs:
        policy.predict(i)
        
print(cache)

In [ ]:
observations = np.stack([obs['obs'] for obs in inputs])
switches = np.stack([obs['switch'] for obs in inputs])
print("Observations shape:", observations.shape)
print("Switches shape:", switches.shape)

In [ ]:
goal_positions = torch.tensor([e.goal_positions for e in envs])
agent_positions = torch.tensor([e.agent_pos for e in envs])
red_goals = goal_positions[:, 0]
green_goals = goal_positions[:, 1]
blue_goals = goal_positions[:, 2]

In [ ]:
# Find the correlation between each act and red_goal.x
def one_hot_encode(arr, maxi=4):
    # Set up one-hot encoding array
    one_hot = np.zeros((*arr.shape, maxi))
    # Populate the one-hot encoding array        
    np.put_along_axis(one_hot, arr[..., None], 1, axis=-1)
    return one_hot

# to_check = goal_positions.flatten(1)
# to_check = one_hot_encode(to_check)
to_check = np.concatenate([
    goal_positions.flatten(1),
    agent_positions], 
    axis=-1)

for name in ("switches.0", "switches.1"):
    act = cache[name]
    corrs = np.corrcoef(act, to_check, rowvar=False)
    # remove the diagonal
    corrs[range(corrs.shape[0]), range(corrs.shape[0])] = 0
    imshow(corrs[32:])

In [ ]:
corr_red_blind = np.corrcoef(cache['switches.0'], goal_positions[:, 0], rowvar=False)
corr_blue_color = np.corrcoef(cache['switches.1'], goal_positions[:, 2], rowvar=False)
corr_red_blind = corr_red_blind[32:, :-2]
corr_blue_color = corr_blue_color[32:, :-2]
corrs = einops.rearrange([corr_red_blind, corr_blue_color], 
                         "type dim neuron -> (dim type) neuron")

imshow(corrs)
imshow(corr_red_blind)
imshow(corr_blue_color)